In [1]:
from riotwatcher import RiotWatcher
import pandas as pd
from converters.data2frames import game_to_dataframe as g2df
from converters.data2frames import game_teams_to_dataframe as gt2df
from converters.data2frames import champs_to_dataframe as ch2df
from converters.data2files import read_json
from config.constants import API_KEY, SLO_GAMES_DIR, SLO_DATASET_CSV, SLO_MATCHES_FILE_PATH, SCRIMS_POSITIONS_COLS
from config.constants import DATA_DRAGON_URL, DD_RUNES_REFORGED, DD_LANGUAGE, DD_RUNES_REFORGED, STATIC_DATA_DIR, CSV_EXPORT_PATH_MERGED
from config.constants import LCK_MATCHES_FILE_PATH, LEAGUES_DATA_DICT, CSV_EXPORT_PATH, EXCEL_EXPORT_PATH, IDS_FILE_PATH, EXPORTS_DIR
import numpy as np
from slds import Slds
import py_gg as cgg


In [2]:
slds = Slds(region='EUW1', league='SOLOQ')

In [3]:
rw = slds.rw

In [4]:
cgg.init('abab11f3c536f617e5ecbeea49aadfc8')

In [6]:
champs = cgg.champions.all()

In [8]:
stats_overall = cgg.statistics.overall()

In [10]:
stats_general = cgg.statistics.general()

In [36]:
champ_40 = cgg.champions.specific(id=40, options={'champData': 'damage'})

In [54]:
champ_40[0]['damageComposition']

{'percentMagical': 0.8188064464955549,
 'percentPhysical': 0.102042692617938,
 'percentTrue': 0.07905633434429414,
 'total': 8015.690826796649,
 'totalMagical': 6563.299322096381,
 'totalPhysical': 817.9426751592357,
 'totalTrue': 633.6911340037275}

In [40]:
champs = ch2df(read_json(save_dir=STATIC_DATA_DIR, file_name='champions'))

In [56]:
champs_damage_percents = {champ[1]['name']:  cgg.champions.specific(id=champ[1]['id'], options={'champData': 'damage'})[0]['damageComposition'] for champ in champs.iterrows()}

In [62]:
pd.DataFrame(champs_damage_percents).T.reset_index().rename(columns={'index': 'champ_name'}).to_excel('champs_dmg_percent.xlsx')

In [ ]:
ml = rw.match.matchlist_by_account(a)

In [ ]:
tl = rw.match.timeline_by_match(match_id=3567214874, region='EUW1')

In [ ]:
fs = tl['frames']

In [ ]:
fs[1]['timestamp'] / 60000

In [ ]:
m = rw.match.by_id(match_id=3366842667, region='EUW1')

In [17]:
df2 = pd.read_csv(LEAGUES_DATA_DICT['SOLOQ'][CSV_EXPORT_PATH_MERGED], index_col=0, encoding='ISO-8859-1')

C:\Users\david\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
df2['lanerole'] = df2['lane'] + '_'  + df2['role']

In [19]:
def position(lanerole):
    if lanerole == 'JUNGLE_NONE':
        return 'JUNG'
    elif lanerole == 'BOTTOM_DUO_SUPPORT':
        return 'SUPP'
    elif lanerole == 'BOTTOM_DUO_CARRY':
        return 'ADC'
    elif lanerole == 'TOP_SOLO':
        return 'TOP'
    elif lanerole == 'MIDDLE_SOLO':
        return 'MID'

In [20]:
df2['position'] = df2.lanerole.apply(position)

In [21]:
df3 = df2[pd.notnull(df2.main_role)]

In [23]:
df4 = df3[df3.position == df3.main_role]

In [34]:
df2.sort_values('position').sort_values('gameId')[['gameId', 'teamId', 'position']]

,gameId,teamId,position
21359,2735302151,200.0,TOP
21357,2735302151,200.0,SUPP
21358,2735302151,200.0,JUNG
21353,2735302151,100.0,None
21352,2735302151,100.0,MID
21350,2735302151,100.0,JUNG
21356,2735302151,200.0,MID
21355,2735302151,200.0,ADC
21354,2735302151,100.0,None
21351,2735302151,100.0,TOP


In [ ]:
df2[df2.lanerole == 'NONE_DUO_SUPPORT'].champ_name

In [ ]:
accs = [23787352, 227994782, 203396130, 231898666, 22654093, 36549969, 33328489, 218694983, 35306044, 36623216, 26609836]

In [ ]:
accs

In [ ]:
pd.DataFrame({'game_id': accs})

In [ ]:
from itertools import chain

In [ ]:
matches = list(chain.from_iterable([rw.match.matchlist_by_account(account_id=acc, end_index=20, region=slds.region, queue=420)['matches'] for acc in accs]))

In [ ]:
gids = list(set([m['gameId'] for m in mls]))

In [ ]:
df1 = pd.read_csv(LEAGUES_DATA_DICT['LCK'][IDS_FILE_PATH], dtype=LEAGUES_DATA_DICT['LCK']['dtypes'])

In [ ]:
list(df1.game_id.map(str) + df1.tournament + df1.hash)

In [ ]:
def something(asd, **kwargs):
    print(kwargs['b'])

In [ ]:
something('asd', b='shit')

In [ ]:
pids = {"participantId":1,"player":{"summonerName":"SKT Untara","profileIcon":4}},{"participantId":2,"player":{"summonerName":"SKT Blossom","profileIcon":7}},{"participantId":3,"player":{"summonerName":"SKT Faker","profileIcon":7}},{"participantId":4,"player":{"summonerName":"SKT Bang","profileIcon":20}},{"participantId":5,"player":{"summonerName":"SKT Wolf","profileIcon":7}},{"participantId":6,"player":{"summonerName":"KZ Khan","profileIcon":6}},{"participantId":7,"player":{"summonerName":"KZ Peanut","profileIcon":6}},{"participantId":8,"player":{"summonerName":"KZ Bdd","profileIcon":6}},{"participantId":9,"player":{"summonerName":"KZ PraY","profileIcon":6}},{"participantId":10,"player":{"summonerName":"KZ GorillA","profileIcon":6}}

In [ ]:
for x in pids:
    print(x)